In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
 !pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 102.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.5 MB/s eta 0:00:00


In [4]:
import sklearn
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
from transformers import DistilBertTokenizerFast
from sklearn.model_selection import train_test_split

In [5]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [6]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Emotion Recognition/training.csv')
eval_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Emotion Recognition/validation.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Emotion Recognition/test.csv')

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16000 non-null  object
 1   label   16000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 250.1+ KB


In [8]:
eval_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2000 non-null   object
 1   label   2000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2000 non-null   object
 1   label   2000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


In [10]:
X_train = train_df['text'].tolist()
y_train = train_df['label'].tolist()

In [11]:
X_eval = eval_df['text'].tolist()
y_eval = eval_df['label'].tolist()

In [12]:
X_test = test_df['text'].tolist()
y_test = test_df['label'].tolist()

In [13]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
eval_encodings = tokenizer(X_eval, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

#### Creating dataset object

In [35]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train))
eval_dataset = tf.data.Dataset.from_tensor_slices((dict(eval_encodings), y_eval))

In [36]:
training_args = TFTrainingArguments(
    output_dir = '/results',
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=1e-5,
    logging_dir = '/logs',
    logging_steps=10,
    eval_steps=100,
)

In [37]:
with training_args.strategy.scope():
  model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)

  trainer = TFTrainer(
      model=model,
      args = training_args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
  )
      
  trainer.train()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use i

In [38]:
result = trainer.evaluate()
result

{'eval_loss': 0.14773468017578126}

In [40]:
# save_directory = '/content/drive/MyDrive/Colab Notebooks/Emotion Recognition'
# model.save_pretrained(save_directory)
# tokenizer.save_pretrained(save_directory)

('/content/drive/MyDrive/Colab Notebooks/Emotion Recognition/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Emotion Recognition/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Emotion Recognition/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/Emotion Recognition/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Emotion Recognition/tokenizer.json')

In [41]:
save_directory = '/content/drive/MyDrive/Colab Notebooks/Emotion Recognition'
tokenizer_fine_tuned = DistilBertTokenizerFast.from_pretrained(save_directory)
model_fine_tuned = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

Some layers from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/Emotion Recognition were not used when initializing TFDistilBertForSequenceClassification: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/Emotion Recognition and are newly initialized: ['dropout_99']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and i

In [42]:
test_encodings = tokenizer(
    X_test,
    truncation=True,
    padding=True,
    return_tensors='tf',
)

In [45]:
testing_data = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test))

In [46]:
final_pred = trainer.predict(testing_data)

In [47]:
final_pred

PredictionOutput(predictions=array([[ 6.8397503, -1.5311292, -2.9173927, -2.0455947, -2.8710191,
        -4.593642 ],
       [ 6.8982596, -1.368342 , -3.1162984, -2.2783005, -2.6191237,
        -4.530299 ],
       [ 6.8502226, -1.5071465, -2.9384503, -2.6830723, -2.4447653,
        -4.298749 ],
       ...,
       [-2.44962  ,  7.230124 , -2.578943 , -2.3294234, -2.6286385,
        -2.5814047],
       [-2.2009795,  7.1231847, -2.7121713, -2.4304626, -1.9478163,
        -3.0387278],
       [-2.598468 , -2.1860037, -4.4090834, -3.4057674,  2.893757 ,
         2.9052048]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 1, 4], dtype=int32), metrics={'eval_loss': 0.14607284545898438})

In [51]:
final_prediction = tf.argmax(final_pred.predictions, axis=1)

In [55]:
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(final_prediction, y_test)
print(cm)
cr = classification_report(final_prediction, y_test)
print(cr)
                           

[[561   3   1  14   6   3]
 [  2 674  37   1   1   7]
 [  0  14 120   0   0   0]
 [ 11   2   1 254   6   0]
 [  7   1   0   6 199  10]
 [  0   1   0   0  12  46]]
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       588
           1       0.97      0.93      0.95       722
           2       0.75      0.90      0.82       134
           3       0.92      0.93      0.93       274
           4       0.89      0.89      0.89       223
           5       0.70      0.78      0.74        59

    accuracy                           0.93      2000
   macro avg       0.87      0.90      0.88      2000
weighted avg       0.93      0.93      0.93      2000

